In [17]:
import pandas as pd
import holidays

In [18]:
# create a date dimension table and populate it 20 years ago to today
date_dim_columns = ['date', 'year', 'month', 'day', 'weekday', 'day_of_year', 'is_holiday', 'is_weekend']
date_dim_df = pd.DataFrame(columns=date_dim_columns)
date_dim_df['date'] = pd.date_range(start='20020101', end='20221231')
date_dim_df['year'] = date_dim_df['date'].dt.year
date_dim_df['month'] = date_dim_df['date'].dt.month
date_dim_df['day'] = date_dim_df['date'].dt.day
date_dim_df['weekday'] = date_dim_df['date'].dt.weekday
date_dim_df['day_of_year'] = date_dim_df['date'].dt.dayofyear
date_dim_df['is_holiday'] = date_dim_df['date'].apply(lambda x: x in holidays.BR(state='SP', years=x.year))
date_dim_df['is_weekend'] = date_dim_df['date'].dt.weekday.isin([5, 6])
date_dim_df.to_csv('date_dim.csv', index=False)